In [1]:
import warnings
warnings.simplefilter('ignore')

import gc

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
pd.set_option('max_columns', 500)
pd.set_option('max_rows', 200)
pd.set_option('float_format', lambda x: '%.6f' % x)

from tqdm import tqdm
tqdm.pandas()

# 合并思路

数据含有三部分，有从属关系，从上至下分为三层：

1. event
2. jet
3. particle

因为同一个 event 的 target 是一样的，因此可以尝试将三层数据合并为一层，用整个 event 来当做输入，进行训练和预测：

1. 将 16 个 particle 合并为一个 jet，合并的顺序为按 abs(particle_px) 从大到小排序，如果超过 16 则截断，小于 16 则补零；
2. 将上一步的 particle data 按 jet_id 与 jet 数据合并；
3. 将 4 个 jet_id 合并为一个 event，合并的顺序为按 abs(jet_px) 从大到小排序，如果超过 4 则截断，小于 4 则补零；
4. 将上一步的 jet data 按 event_id 与 event 数据合并。

上面所述的 4 和 16 均为初步验证设置，后期可以尝试不同的数量。

In [2]:
n_particles = 20
n_jets = 4

In [3]:
# 先读入 particle 数据

df = pd.read_pickle('train_particle_32cols.pickle')

In [4]:
# 按 abs(particle_px) 从大到小将每个 jet_id 里面的 particle 数据进行排序

df['particle_px_abs'] = np.abs(df['particle_px'])
df = df.sort_index(by=['jet_id', 'particle_px_abs'], ascending=False).reset_index()

df = df.drop(['particle_px_abs'], axis=1)

df.head()

,index,particle_category,particle_px,particle_py,particle_pz,particle_energy,particle_mass,particle_distance,x_div_dist,energy_x,mass_x,angle_xy,angle_xz,particle_speed,speed_x,time_x,jet_id_mass_std,jet_id_energy_std,jet_id_mass_x_std,jet_id_energy_x_std,jet_id_x_div_dist_std,jet_id_speed_std,jet_id_speed_x_std,jet_id_px_std,jet_id_angle_xy_std,jet_id_angle_xz_std,jet_id_time_x_std,particle_category_abs,particle_category_unique_len,particle_category_unique_len_abs,jet_id_energy_sum,jet_id_mass_sum,jet_id
0,5385914,321,33.437500,-0.466797,0.932129,33.437500,0.493652,33.437500,0.999512,33.437500,0.493408,-89.187500,88.375000,11.640625,11.632812,2.873047,0.336426,11.085938,0.335693,11.078125,0.011742,5.187500,5.191406,11.078125,87.000000,85.812500,1.114258,321,8,5,338.500000,9.054688,fffff8fef65647bf964e2e521f8df138
1,5445737,-211,32.281250,0.308594,0.720703,32.312500,0.139526,32.312500,0.999512,32.281250,0.139526,89.437500,88.750000,21.515625,21.515625,1.500977,0.336426,11.085938,0.335693,11.078125,0.011742,5.187500,5.191406,11.078125,87.000000,85.812500,1.114258,211,8,5,338.500000,9.054688,fffff8fef65647bf964e2e521f8df138
2,5609036,-321,28.218750,-0.598633,1.284180,28.265625,0.493652,28.265625,0.998535,28.234375,0.493164,-88.812500,87.375000,10.703125,10.687500,2.640625,0.336426,11.085938,0.335693,11.078125,0.011742,5.187500,5.191406,11.078125,87.000000,85.812500,1.114258,321,8,5,338.500000,9.054688,fffff8fef65647bf964e2e521f8df138
3,5431519,22,28.171875,0.832031,1.408203,28.218750,0.000000,28.218750,0.998535,28.171875,0.000000,88.312500,87.125000,0.000000,0.000000,0.000000,0.336426,11.085938,0.335693,11.078125,0.011742,5.187500,5.191406,11.078125,87.000000,85.812500,1.114258,22,8,5,338.500000,9.054688,fffff8fef65647bf964e2e521f8df138
4,5646787,321,28.078125,-0.428955,1.544922,28.140625,0.493652,28.125000,0.998535,28.093750,0.492920,-89.125000,86.875000,10.679688,10.656250,2.634766,0.336426,11.085938,0.335693,11.078125,0.011742,5.187500,5.191406,11.078125,87.000000,85.812500,1.114258,321,8,5,338.500000,9.054688,fffff8fef65647bf964e2e521f8df138


In [5]:
# 生成一些辅助变量
# di 记录的是 {jet_id: [当前 jet_id 已加入的 particle 个数, 当前 jet_id 的序号]}

jet = df.jet_id.unique()
num_total_jet = jet.shape[0]

di = dict()
for i, j in tqdm(enumerate(jet.tolist())):
    di[j] = [0, i]

1134555it [00:02, 471652.27it/s]


In [6]:
common_cols = ['jet_id_mass_std', 'jet_id_mass_x_std', 'jet_id_energy_x_std', 
               'jet_id_x_div_dist_std', 'jet_id_speed_std', 'jet_id_speed_x_std', 'jet_id_px_std',
               'jet_id_angle_xy_std', 'jet_id_angle_xz_std', 'jet_id_time_x_std',
               'particle_category_unique_len', 'particle_category_unique_len_abs', 
               'jet_id_energy_sum', 'jet_id_mass_sum']

particle_cols = ['particle_category', 'particle_category_abs', 'particle_px', 'particle_py', 'particle_pz',
                 'particle_energy', 'particle_mass', 'x_div_dist', 'particle_distance', 'particle_speed',
                 'speed_x', 'energy_x', 'mass_x', 'time_x', 'angle_xy', 'angle_xz']

useless_cols = ['index', 'jet_id']

col_shape = n_particles * len(particle_cols) + len(common_cols)

In [7]:
df = df.reindex(columns=common_cols+particle_cols+useless_cols)
df.head()

,jet_id_mass_std,jet_id_mass_x_std,jet_id_energy_x_std,jet_id_x_div_dist_std,jet_id_speed_std,jet_id_speed_x_std,jet_id_px_std,jet_id_angle_xy_std,jet_id_angle_xz_std,jet_id_time_x_std,particle_category_unique_len,particle_category_unique_len_abs,jet_id_energy_sum,jet_id_mass_sum,particle_category,particle_category_abs,particle_px,particle_py,particle_pz,particle_energy,particle_mass,x_div_dist,particle_distance,particle_speed,speed_x,energy_x,mass_x,time_x,angle_xy,angle_xz,index,jet_id
0,0.336426,0.335693,11.078125,0.011742,5.187500,5.191406,11.078125,87.000000,85.812500,1.114258,8,5,338.500000,9.054688,321,321,33.437500,-0.466797,0.932129,33.437500,0.493652,0.999512,33.437500,11.640625,11.632812,33.437500,0.493408,2.873047,-89.187500,88.375000,5385914,fffff8fef65647bf964e2e521f8df138
1,0.336426,0.335693,11.078125,0.011742,5.187500,5.191406,11.078125,87.000000,85.812500,1.114258,8,5,338.500000,9.054688,-211,211,32.281250,0.308594,0.720703,32.312500,0.139526,0.999512,32.312500,21.515625,21.515625,32.281250,0.139526,1.500977,89.437500,88.750000,5445737,fffff8fef65647bf964e2e521f8df138
2,0.336426,0.335693,11.078125,0.011742,5.187500,5.191406,11.078125,87.000000,85.812500,1.114258,8,5,338.500000,9.054688,-321,321,28.218750,-0.598633,1.284180,28.265625,0.493652,0.998535,28.265625,10.703125,10.687500,28.234375,0.493164,2.640625,-88.812500,87.375000,5609036,fffff8fef65647bf964e2e521f8df138
3,0.336426,0.335693,11.078125,0.011742,5.187500,5.191406,11.078125,87.000000,85.812500,1.114258,8,5,338.500000,9.054688,22,22,28.171875,0.832031,1.408203,28.218750,0.000000,0.998535,28.218750,0.000000,0.000000,28.171875,0.000000,0.000000,88.312500,87.125000,5431519,fffff8fef65647bf964e2e521f8df138
4,0.336426,0.335693,11.078125,0.011742,5.187500,5.191406,11.078125,87.000000,85.812500,1.114258,8,5,338.500000,9.054688,321,321,28.078125,-0.428955,1.544922,28.140625,0.493652,0.998535,28.125000,10.679688,10.656250,28.093750,0.492920,2.634766,-89.125000,86.875000,5646787,fffff8fef65647bf964e2e521f8df138


In [8]:
for i, col in enumerate(df.columns):
    print(i, col)

0 jet_id_mass_std
1 jet_id_mass_x_std
2 jet_id_energy_x_std
3 jet_id_x_div_dist_std
4 jet_id_speed_std
5 jet_id_speed_x_std
6 jet_id_px_std
7 jet_id_angle_xy_std
8 jet_id_angle_xz_std
9 jet_id_time_x_std
10 particle_category_unique_len
11 particle_category_unique_len_abs
12 jet_id_energy_sum
13 jet_id_mass_sum
14 particle_category
15 particle_category_abs
16 particle_px
17 particle_py
18 particle_pz
19 particle_energy
20 particle_mass
21 x_div_dist
22 particle_distance
23 particle_speed
24 speed_x
25 energy_x
26 mass_x
27 time_x
28 angle_xy
29 angle_xz
30 index
31 jet_id


In [9]:
# 用 np.ndarry 来做处理，使用 dataframe 会相当慢

particle_data = np.zeros(shape=(num_total_jet, col_shape), dtype=np.float16)
len_commons = len(common_cols)
len_particles = len(particle_cols)

for row in tqdm(df.values):
    # 取出该行 particle 的 jet_id
    jet_id = row[-1]
    # n: 当前 jet_id 已加入的 particle 个数
    # i: 当前 jet_id 的序号
    n, i = di[jet_id]
    # 如果当前 jet_id 的 particle 个数少于 n_particles 则加入进来
    if n < n_particles:
        # 首先是 common cols, 只在第一次时添加
        if n == 0:
            particle_data[i, 0:len_commons+len_particles] = row[0:len_commons+len_particles]
        else:
            particle_data[i, len_commons+n*len_particles:len_commons+(n+1)*len_particles] = row[len_commons:len_commons+len_particles]
        # particle 个数加 1
        di[jet_id][0] += 1

100%|██████████| 24297352/24297352 [01:18<00:00, 308015.76it/s]


In [10]:
del df
gc.collect()

40

In [11]:
# 合并完的 particle 数据

df_particle = pd.DataFrame(particle_data)

In [12]:
df_particle.columns = common_cols + [f'p{i}_{j}' for i in range(n_particles) for j in particle_cols]
df_particle['jet_id'] = jet

In [13]:
df_particle.head()

,jet_id_mass_std,jet_id_mass_x_std,jet_id_energy_x_std,jet_id_x_div_dist_std,jet_id_speed_std,jet_id_speed_x_std,jet_id_px_std,jet_id_angle_xy_std,jet_id_angle_xz_std,jet_id_time_x_std,particle_category_unique_len,particle_category_unique_len_abs,jet_id_energy_sum,jet_id_mass_sum,p0_particle_category,p0_particle_category_abs,p0_particle_px,p0_particle_py,p0_particle_pz,p0_particle_energy,p0_particle_mass,p0_x_div_dist,p0_particle_distance,p0_particle_speed,p0_speed_x,p0_energy_x,p0_mass_x,p0_time_x,p0_angle_xy,p0_angle_xz,p1_particle_category,p1_particle_category_abs,p1_particle_px,p1_particle_py,p1_particle_pz,p1_particle_energy,p1_particle_mass,p1_x_div_dist,p1_particle_distance,p1_particle_speed,p1_speed_x,p1_energy_x,p1_mass_x,p1_time_x,p1_angle_xy,p1_angle_xz,p2_particle_category,p2_particle_category_abs,p2_particle_px,p2_particle_py,p2_particle_pz,p2_particle_energy,p2_particle_mass,p2_x_div_dist,p2_particle_distance,p2_particle_speed,p2_speed_x,p2_energy_x,p2_mass_x,p2_time_x,p2_angle_xy,p2_angle_xz,p3_particle_category,p3_particle_category_abs,p3_particle_px,p3_particle_py,p3_particle_pz,p3_particle_energy,p3_particle_mass,p3_x_div_dist,p3_particle_distance,p3_particle_speed,p3_speed_x,p3_energy_x,p3_mass_x,p3_time_x,p3_angle_xy,p3_angle_xz,p4_particle_category,p4_particle_category_abs,p4_particle_px,p4_particle_py,p4_particle_pz,p4_particle_energy,p4_particle_mass,p4_x_div_dist,p4_particle_distance,p4_particle_speed,p4_speed_x,p4_energy_x,p4_mass_x,p4_time_x,p4_angle_xy,p4_angle_xz,p5_particle_category,p5_particle_category_abs,p5_particle_px,p5_particle_py,p5_particle_pz,p5_particle_energy,p5_particle_mass,p5_x_div_dist,p5_particle_distance,p5_particle_speed,p5_speed_x,p5_energy_x,p5_mass_x,p5_time_x,p5_angle_xy,p5_angle_xz,p6_particle_category,p6_particle_category_abs,p6_particle_px,p6_particle_py,p6_particle_pz,p6_particle_energy,p6_particle_mass,p6_x_div_dist,p6_particle_distance,p6_particle_speed,p6_speed_x,p6_energy_x,p6_mass_x,p6_time_x,p6_angle_xy,p6_angle_xz,p7_particle_category,p7_particle_category_abs,p7_particle_px,p7_particle_py,p7_particle_pz,p7_particle_energy,p7_particle_mass,p7_x_div_dist,p7_particle_distance,p7_particle_speed,p7_speed_x,p7_energy_x,p7_mass_x,p7_time_x,p7_angle_xy,p7_angle_xz,p8_particle_category,p8_particle_category_abs,p8_particle_px,p8_particle_py,p8_particle_pz,p8_particle_energy,p8_particle_mass,p8_x_div_dist,p8_particle_distance,p8_particle_speed,p8_speed_x,p8_energy_x,p8_mass_x,p8_time_x,p8_angle_xy,p8_angle_xz,p9_particle_category,p9_particle_category_abs,p9_particle_px,p9_particle_py,p9_particle_pz,p9_particle_energy,p9_particle_mass,p9_x_div_dist,p9_particle_distance,p9_particle_speed,p9_speed_x,p9_energy_x,p9_mass_x,p9_time_x,p9_angle_xy,p9_angle_xz,p10_particle_category,p10_particle_category_abs,p10_particle_px,p10_particle_py,p10_particle_pz,p10_particle_energy,p10_particle_mass,p10_x_div_dist,p10_particle_distance,p10_particle_speed,p10_speed_x,p10_energy_x,p10_mass_x,p10_time_x,p10_angle_xy,p10_angle_xz,p11_particle_category,p11_particle_category_abs,p11_particle_px,p11_particle_py,p11_particle_pz,p11_particle_energy,p11_particle_mass,p11_x_div_dist,p11_particle_distance,p11_particle_speed,p11_speed_x,p11_energy_x,p11_mass_x,p11_time_x,p11_angle_xy,p11_angle_xz,p12_particle_category,p12_particle_category_abs,p12_particle_px,p12_particle_py,p12_particle_pz,p12_particle_energy,p12_particle_mass,p12_x_div_dist,p12_particle_distance,p12_particle_speed,p12_speed_x,p12_energy_x,p12_mass_x,p12_time_x,p12_angle_xy,p12_angle_xz,p13_particle_category,p13_particle_category_abs,p13_particle_px,p13_particle_py,p13_particle_pz,p13_particle_energy,p13_particle_mass,p13_x_div_dist,p13_particle_distance,p13_particle_speed,p13_speed_x,p13_energy_x,p13_mass_x,p13_time_x,p13_angle_xy,p13_angle_xz,p14_particle_category,p14_particle_category_abs,p14_particle_px,p14_particle_py,p14_particle_pz,p14_particle_energy,p14_particle_mass,p14_x_div_dist,p14_particle_distance,p14_particle_speed,p14_spe

In [14]:
df_particle['particle_category_unique_len'] = df_particle['particle_category_unique_len'].astype('int')
df_particle['particle_category_unique_len_abs'] = df_particle['particle_category_unique_len_abs'].astype('int')

for i in tqdm(range(n_particles)):
    df_particle[f'p{i}_particle_category'] = df_particle[f'p{i}_particle_category'].astype('int')
    df_particle[f'p{i}_particle_category_abs'] = df_particle[f'p{i}_particle_category_abs'].astype('int')

100%|██████████| 20/20 [00:05<00:00,  3.55it/s]


In [15]:
# 读入 jet 数据

# train_jet = pd.read_csv('jet_complex_data/complex_train_R04_jet.csv')
train_jet = pd.read_pickle('train_jet.float16.pickle')

In [16]:
# particle 数据与 jet 数据合并

df = pd.merge(train_jet, df_particle, on=['jet_id'])

In [17]:
del df_particle, particle_data
gc.collect()

60

In [18]:
df.head()

,event_id,jet_energy,jet_id,jet_mass,jet_px,jet_py,jet_pz,label,number_of_particles_in_this_jet,jet_distance,xy_dis,yz_dis,zx_dis,x_div_dist,y_div_dist,z_div_dist,xy_div_dist,yz_div_dist,zx_div_dist,energy_x,energy_y,energy_z,energy_xy,energy_yz,energy_zx,mass_x,mass_y,mass_z,mass_xy,mass_yz,mass_zx,angle_xy,angle_yx,angle_yz,angle_zy,angle_zx,angle_xz,mean_particles_mass,mean_particles_energy,jet_speed,speed_x,speed_y,speed_z,speed_xy,speed_yz,speed_zx,time_dis,time_x,time_y,time_z,time_xy,time_yz,time_zx,jet_mv,mv_x,mv_y,mv_z,mv_xy,mv_yz,mv_zx,particle_mv,particle_mv_x,particle_mv_y,particle_mv_z,particle_mv_xy,particle_mv_yz,particle_mv_zx,event_id_count,event_id_number_particles_max,event_id_number_particles_mean,event_id_number_particles_min,event_id_number_particles_std,event_id_mass_max,event_id_mass_mean,event_id_mass_min,event_id_mass_std,event_id_energy_max,event_id_energy_mean,event_id_energy_min,event_id_energy_std,event_id_mass_x_max,event_id_mass_x_mean,event_id_mass_x_min,event_id_mass_x_std,event_id_mass_y_max,event_id_mass_y_mean,event_id_mass_y_min,event_id_mass_y_std,event_id_mass_z_max,event_id_mass_z_mean,event_id_mass_z_min,event_id_mass_z_std,event_id_mass_xy_max,event_id_mass_xy_mean,event_id_mass_xy_min,event_id_mass_xy_std,event_id_mass_yz_max,event_id_mass_yz_mean,event_id_mass_yz_min,event_id_mass_yz_std,event_id_mass_zx_max,event_id_mass_zx_mean,event_id_mass_zx_min,event_id_mass_zx_std,event_id_energy_x_max,event_id_energy_x_mean,event_id_energy_x_min,event_id_energy_x_std,event_id_energy_y_max,event_id_energy_y_mean,event_id_energy_y_min,event_id_energy_y_std,event_id_energy_z_max,event_id_energy_z_mean,event_id_energy_z_min,event_id_energy_z_std,event_id_energy_xy_max,event_id_energy_xy_mean,event_id_energy_xy_min,event_id_energy_xy_std,event_id_energy_yz_max,event_id_energy_yz_mean,event_id_energy_yz_min,event_id_energy_yz_std,event_id_energy_zx_max,event_id_energy_zx_mean,event_id_energy_zx_min,event_id_energy_zx_std,event_id_particles_mass_max,event_id_particles_mass_mean,event_id_particles_mass_min,event_id_particles_mass_std,event_id_particles_energy_max,event_id_particles_energy_mean,event_id_particles_energy_min,event_id_particles_energy_std,event_id_distance_max,event_id_distance_mean,event_id_distance_min,event_id_distance_std,event_id_xy_dis_max,event_id_xy_dis_mean,event_id_xy_dis_min,event_id_xy_dis_std,event_id_yz_dis_max,event_id_yz_dis_mean,event_id_yz_dis_min,event_id_yz_dis_std,event_id_zx_dis_max,event_id_zx_dis_mean,event_id_zx_dis_min,event_id_zx_dis_std,event_id_x_div_dist_max,event_id_x_div_dist_mean,event_id_x_div_dist_min,event_id_x_div_dist_std,event_id_y_div_dist_max,event_id_y_div_dist_mean,event_id_y_div_dist_min,event_id_y_div_dist_std,event_id_z_div_dist_max,event_id_z_div_dist_mean,event_id_z_div_dist_min,event_id_z_div_dist_std,event_id_xy_div_dist_max,event_id_xy_div_dist_mean,event_id_xy_div_dist_min,event_id_xy_div_dist_std,event_id_yz_div_dist_max,event_id_yz_div_dist_mean,event_id_yz_div_dist_min,event_id_yz_div_dist_std,event_id_zx_div_dist_max,event_id_zx_div_dist_mean,event_id_zx_div_dist_min,event_id_zx_div_dist_std,event_id_speed_max,event_id_speed_mean,event_id_speed_min,event_id_speed_std,event_id_speed_x_max,event_id_speed_x_mean,event_id_speed_x_min,event_id_speed_x_std,event_id_speed_y_max,event_id_speed_y_mean,event_id_speed_y_min,event_id_speed_y_std,event_id_speed_z_max,event_id_speed_z_mean,event_id_speed_z_min,event_id_speed_z_std,event_id_speed_xy_max,event_id_speed_xy_mean,event_id_speed_xy_min,event_id_speed_xy_std,event_id_speed_yz_max,event_id_speed_yz_mean,event_id_speed_yz_min,event_id_speed_yz_std,event_id_speed_zx_max,event_id_speed_zx_mean,event_id_speed_zx_min,event_id_speed_zx_std,event_id_px_max,event_id_px_mean,event_id_px_min,event_id_px_std,event_id_py_max,event_id_py_mean,event_id_py_min,event_id_py_std,event_id_pz_max,event_id_pz_mean,event_id_pz_min,event_id_pz_std,event_id_angle_xy_max,event_id_angle_xy_mean,event_id_angle_x

In [21]:
for col in (df.columns.tolist()):
    try:
        if df[col].dtypes == 'float64':
            df[col] = df[col].astype('float16')
    except:
        print(f'error on convert: {col}')

In [22]:
gc.collect()

40

In [23]:
# 按 abs(jet_px) 从大到小将每个 event_id 里面的 jet 数据进行排序

df['jet_px_abs'] = np.abs(df['jet_px'])
df = df.sort_index(by=['event_id', 'jet_px_abs'], ascending=False).reset_index()

df = df.drop(['jet_px_abs'], axis=1)

In [24]:
df.head()

,index,event_id,jet_energy,jet_id,jet_mass,jet_px,jet_py,jet_pz,label,number_of_particles_in_this_jet,jet_distance,xy_dis,yz_dis,zx_dis,x_div_dist,y_div_dist,z_div_dist,xy_div_dist,yz_div_dist,zx_div_dist,energy_x,energy_y,energy_z,energy_xy,energy_yz,energy_zx,mass_x,mass_y,mass_z,mass_xy,mass_yz,mass_zx,angle_xy,angle_yx,angle_yz,angle_zy,angle_zx,angle_xz,mean_particles_mass,mean_particles_energy,jet_speed,speed_x,speed_y,speed_z,speed_xy,speed_yz,speed_zx,time_dis,time_x,time_y,time_z,time_xy,time_yz,time_zx,jet_mv,mv_x,mv_y,mv_z,mv_xy,mv_yz,mv_zx,particle_mv,particle_mv_x,particle_mv_y,particle_mv_z,particle_mv_xy,particle_mv_yz,particle_mv_zx,event_id_count,event_id_number_particles_max,event_id_number_particles_mean,event_id_number_particles_min,event_id_number_particles_std,event_id_mass_max,event_id_mass_mean,event_id_mass_min,event_id_mass_std,event_id_energy_max,event_id_energy_mean,event_id_energy_min,event_id_energy_std,event_id_mass_x_max,event_id_mass_x_mean,event_id_mass_x_min,event_id_mass_x_std,event_id_mass_y_max,event_id_mass_y_mean,event_id_mass_y_min,event_id_mass_y_std,event_id_mass_z_max,event_id_mass_z_mean,event_id_mass_z_min,event_id_mass_z_std,event_id_mass_xy_max,event_id_mass_xy_mean,event_id_mass_xy_min,event_id_mass_xy_std,event_id_mass_yz_max,event_id_mass_yz_mean,event_id_mass_yz_min,event_id_mass_yz_std,event_id_mass_zx_max,event_id_mass_zx_mean,event_id_mass_zx_min,event_id_mass_zx_std,event_id_energy_x_max,event_id_energy_x_mean,event_id_energy_x_min,event_id_energy_x_std,event_id_energy_y_max,event_id_energy_y_mean,event_id_energy_y_min,event_id_energy_y_std,event_id_energy_z_max,event_id_energy_z_mean,event_id_energy_z_min,event_id_energy_z_std,event_id_energy_xy_max,event_id_energy_xy_mean,event_id_energy_xy_min,event_id_energy_xy_std,event_id_energy_yz_max,event_id_energy_yz_mean,event_id_energy_yz_min,event_id_energy_yz_std,event_id_energy_zx_max,event_id_energy_zx_mean,event_id_energy_zx_min,event_id_energy_zx_std,event_id_particles_mass_max,event_id_particles_mass_mean,event_id_particles_mass_min,event_id_particles_mass_std,event_id_particles_energy_max,event_id_particles_energy_mean,event_id_particles_energy_min,event_id_particles_energy_std,event_id_distance_max,event_id_distance_mean,event_id_distance_min,event_id_distance_std,event_id_xy_dis_max,event_id_xy_dis_mean,event_id_xy_dis_min,event_id_xy_dis_std,event_id_yz_dis_max,event_id_yz_dis_mean,event_id_yz_dis_min,event_id_yz_dis_std,event_id_zx_dis_max,event_id_zx_dis_mean,event_id_zx_dis_min,event_id_zx_dis_std,event_id_x_div_dist_max,event_id_x_div_dist_mean,event_id_x_div_dist_min,event_id_x_div_dist_std,event_id_y_div_dist_max,event_id_y_div_dist_mean,event_id_y_div_dist_min,event_id_y_div_dist_std,event_id_z_div_dist_max,event_id_z_div_dist_mean,event_id_z_div_dist_min,event_id_z_div_dist_std,event_id_xy_div_dist_max,event_id_xy_div_dist_mean,event_id_xy_div_dist_min,event_id_xy_div_dist_std,event_id_yz_div_dist_max,event_id_yz_div_dist_mean,event_id_yz_div_dist_min,event_id_yz_div_dist_std,event_id_zx_div_dist_max,event_id_zx_div_dist_mean,event_id_zx_div_dist_min,event_id_zx_div_dist_std,event_id_speed_max,event_id_speed_mean,event_id_speed_min,event_id_speed_std,event_id_speed_x_max,event_id_speed_x_mean,event_id_speed_x_min,event_id_speed_x_std,event_id_speed_y_max,event_id_speed_y_mean,event_id_speed_y_min,event_id_speed_y_std,event_id_speed_z_max,event_id_speed_z_mean,event_id_speed_z_min,event_id_speed_z_std,event_id_speed_xy_max,event_id_speed_xy_mean,event_id_speed_xy_min,event_id_speed_xy_std,event_id_speed_yz_max,event_id_speed_yz_mean,event_id_speed_yz_min,event_id_speed_yz_std,event_id_speed_zx_max,event_id_speed_zx_mean,event_id_speed_zx_min,event_id_speed_zx_std,event_id_px_max,event_id_px_mean,event_id_px_min,event_id_px_std,event_id_py_max,event_id_py_mean,event_id_py_min,event_id_py_std,event_id_pz_max,event_id_pz_mean,event_id_pz_min,event_id_pz_std,event_id_angle_xy_max,event_id_angle_xy_mean,event_id_a

In [25]:
# 需要合并的特征

cols = [col for col in df.columns.tolist() if col not in ['index', 'jet_id', 'event_id']]

In [26]:
df = df.reindex(columns=cols+['index', 'jet_id', 'event_id'])

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1134555 entries, 0 to 1134554
Columns: 662 entries, jet_energy to event_id
dtypes: float16(613), int16(4), int64(43), object(2)
memory usage: 1.7+ GB


In [28]:
df.head()

,jet_energy,jet_mass,jet_px,jet_py,jet_pz,label,number_of_particles_in_this_jet,jet_distance,xy_dis,yz_dis,zx_dis,x_div_dist,y_div_dist,z_div_dist,xy_div_dist,yz_div_dist,zx_div_dist,energy_x,energy_y,energy_z,energy_xy,energy_yz,energy_zx,mass_x,mass_y,mass_z,mass_xy,mass_yz,mass_zx,angle_xy,angle_yx,angle_yz,angle_zy,angle_zx,angle_xz,mean_particles_mass,mean_particles_energy,jet_speed,speed_x,speed_y,speed_z,speed_xy,speed_yz,speed_zx,time_dis,time_x,time_y,time_z,time_xy,time_yz,time_zx,jet_mv,mv_x,mv_y,mv_z,mv_xy,mv_yz,mv_zx,particle_mv,particle_mv_x,particle_mv_y,particle_mv_z,particle_mv_xy,particle_mv_yz,particle_mv_zx,event_id_count,event_id_number_particles_max,event_id_number_particles_mean,event_id_number_particles_min,event_id_number_particles_std,event_id_mass_max,event_id_mass_mean,event_id_mass_min,event_id_mass_std,event_id_energy_max,event_id_energy_mean,event_id_energy_min,event_id_energy_std,event_id_mass_x_max,event_id_mass_x_mean,event_id_mass_x_min,event_id_mass_x_std,event_id_mass_y_max,event_id_mass_y_mean,event_id_mass_y_min,event_id_mass_y_std,event_id_mass_z_max,event_id_mass_z_mean,event_id_mass_z_min,event_id_mass_z_std,event_id_mass_xy_max,event_id_mass_xy_mean,event_id_mass_xy_min,event_id_mass_xy_std,event_id_mass_yz_max,event_id_mass_yz_mean,event_id_mass_yz_min,event_id_mass_yz_std,event_id_mass_zx_max,event_id_mass_zx_mean,event_id_mass_zx_min,event_id_mass_zx_std,event_id_energy_x_max,event_id_energy_x_mean,event_id_energy_x_min,event_id_energy_x_std,event_id_energy_y_max,event_id_energy_y_mean,event_id_energy_y_min,event_id_energy_y_std,event_id_energy_z_max,event_id_energy_z_mean,event_id_energy_z_min,event_id_energy_z_std,event_id_energy_xy_max,event_id_energy_xy_mean,event_id_energy_xy_min,event_id_energy_xy_std,event_id_energy_yz_max,event_id_energy_yz_mean,event_id_energy_yz_min,event_id_energy_yz_std,event_id_energy_zx_max,event_id_energy_zx_mean,event_id_energy_zx_min,event_id_energy_zx_std,event_id_particles_mass_max,event_id_particles_mass_mean,event_id_particles_mass_min,event_id_particles_mass_std,event_id_particles_energy_max,event_id_particles_energy_mean,event_id_particles_energy_min,event_id_particles_energy_std,event_id_distance_max,event_id_distance_mean,event_id_distance_min,event_id_distance_std,event_id_xy_dis_max,event_id_xy_dis_mean,event_id_xy_dis_min,event_id_xy_dis_std,event_id_yz_dis_max,event_id_yz_dis_mean,event_id_yz_dis_min,event_id_yz_dis_std,event_id_zx_dis_max,event_id_zx_dis_mean,event_id_zx_dis_min,event_id_zx_dis_std,event_id_x_div_dist_max,event_id_x_div_dist_mean,event_id_x_div_dist_min,event_id_x_div_dist_std,event_id_y_div_dist_max,event_id_y_div_dist_mean,event_id_y_div_dist_min,event_id_y_div_dist_std,event_id_z_div_dist_max,event_id_z_div_dist_mean,event_id_z_div_dist_min,event_id_z_div_dist_std,event_id_xy_div_dist_max,event_id_xy_div_dist_mean,event_id_xy_div_dist_min,event_id_xy_div_dist_std,event_id_yz_div_dist_max,event_id_yz_div_dist_mean,event_id_yz_div_dist_min,event_id_yz_div_dist_std,event_id_zx_div_dist_max,event_id_zx_div_dist_mean,event_id_zx_div_dist_min,event_id_zx_div_dist_std,event_id_speed_max,event_id_speed_mean,event_id_speed_min,event_id_speed_std,event_id_speed_x_max,event_id_speed_x_mean,event_id_speed_x_min,event_id_speed_x_std,event_id_speed_y_max,event_id_speed_y_mean,event_id_speed_y_min,event_id_speed_y_std,event_id_speed_z_max,event_id_speed_z_mean,event_id_speed_z_min,event_id_speed_z_std,event_id_speed_xy_max,event_id_speed_xy_mean,event_id_speed_xy_min,event_id_speed_xy_std,event_id_speed_yz_max,event_id_speed_yz_mean,event_id_speed_yz_min,event_id_speed_yz_std,event_id_speed_zx_max,event_id_speed_zx_mean,event_id_speed_zx_min,event_id_speed_zx_std,event_id_px_max,event_id_px_mean,event_id_px_min,event_id_px_std,event_id_py_max,event_id_py_mean,event_id_py_min,event_id_py_std,event_id_pz_max,event_id_pz_mean,event_id_pz_min,event_id_pz_std,event_id_angle_xy_max,event_id_angle_xy_mean,event_id_angle_xy_min,event_id_a

In [29]:
df.to_pickle('train_jet_temp.pickle')

# 在这里重启并往下执行

In [1]:
import warnings
warnings.simplefilter('ignore')

import gc

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
pd.set_option('max_columns', 500)
pd.set_option('max_rows', 200)
pd.set_option('float_format', lambda x: '%.6f' % x)

from tqdm import tqdm
tqdm.pandas()

In [2]:
n_particles = 20
n_jets = 4

In [3]:
df = pd.read_pickle('train_jet_temp.pickle')

In [5]:
# 生成一些辅助变量
# di 记录的是 {event_id: [当前 event_id 已加入的 jet 个数, 当前 event_id 的序号]}

event = df.event_id.unique()
num_total_event = event.shape[0]

di = dict()
for i, j in tqdm(enumerate(event.tolist())):
    di[j] = [0, i]

371377it [00:00, 502377.95it/s]


In [6]:
common_cols = [col for col in df.columns.tolist() if col.startswith('event_id_')]
useless_cols = ['index', 'jet_id', 'event_id']
jet_cols = [col for col in df.columns.tolist() if col not in common_cols+useless_cols]

col_shape = n_jets * len(jet_cols) + len(common_cols)

In [7]:
df = df.reindex(columns=common_cols+jet_cols+useless_cols)
df.head()

,event_id_count,event_id_number_particles_max,event_id_number_particles_mean,event_id_number_particles_min,event_id_number_particles_std,event_id_mass_max,event_id_mass_mean,event_id_mass_min,event_id_mass_std,event_id_energy_max,event_id_energy_mean,event_id_energy_min,event_id_energy_std,event_id_mass_x_max,event_id_mass_x_mean,event_id_mass_x_min,event_id_mass_x_std,event_id_mass_y_max,event_id_mass_y_mean,event_id_mass_y_min,event_id_mass_y_std,event_id_mass_z_max,event_id_mass_z_mean,event_id_mass_z_min,event_id_mass_z_std,event_id_mass_xy_max,event_id_mass_xy_mean,event_id_mass_xy_min,event_id_mass_xy_std,event_id_mass_yz_max,event_id_mass_yz_mean,event_id_mass_yz_min,event_id_mass_yz_std,event_id_mass_zx_max,event_id_mass_zx_mean,event_id_mass_zx_min,event_id_mass_zx_std,event_id_energy_x_max,event_id_energy_x_mean,event_id_energy_x_min,event_id_energy_x_std,event_id_energy_y_max,event_id_energy_y_mean,event_id_energy_y_min,event_id_energy_y_std,event_id_energy_z_max,event_id_energy_z_mean,event_id_energy_z_min,event_id_energy_z_std,event_id_energy_xy_max,event_id_energy_xy_mean,event_id_energy_xy_min,event_id_energy_xy_std,event_id_energy_yz_max,event_id_energy_yz_mean,event_id_energy_yz_min,event_id_energy_yz_std,event_id_energy_zx_max,event_id_energy_zx_mean,event_id_energy_zx_min,event_id_energy_zx_std,event_id_particles_mass_max,event_id_particles_mass_mean,event_id_particles_mass_min,event_id_particles_mass_std,event_id_particles_energy_max,event_id_particles_energy_mean,event_id_particles_energy_min,event_id_particles_energy_std,event_id_distance_max,event_id_distance_mean,event_id_distance_min,event_id_distance_std,event_id_xy_dis_max,event_id_xy_dis_mean,event_id_xy_dis_min,event_id_xy_dis_std,event_id_yz_dis_max,event_id_yz_dis_mean,event_id_yz_dis_min,event_id_yz_dis_std,event_id_zx_dis_max,event_id_zx_dis_mean,event_id_zx_dis_min,event_id_zx_dis_std,event_id_x_div_dist_max,event_id_x_div_dist_mean,event_id_x_div_dist_min,event_id_x_div_dist_std,event_id_y_div_dist_max,event_id_y_div_dist_mean,event_id_y_div_dist_min,event_id_y_div_dist_std,event_id_z_div_dist_max,event_id_z_div_dist_mean,event_id_z_div_dist_min,event_id_z_div_dist_std,event_id_xy_div_dist_max,event_id_xy_div_dist_mean,event_id_xy_div_dist_min,event_id_xy_div_dist_std,event_id_yz_div_dist_max,event_id_yz_div_dist_mean,event_id_yz_div_dist_min,event_id_yz_div_dist_std,event_id_zx_div_dist_max,event_id_zx_div_dist_mean,event_id_zx_div_dist_min,event_id_zx_div_dist_std,event_id_speed_max,event_id_speed_mean,event_id_speed_min,event_id_speed_std,event_id_speed_x_max,event_id_speed_x_mean,event_id_speed_x_min,event_id_speed_x_std,event_id_speed_y_max,event_id_speed_y_mean,event_id_speed_y_min,event_id_speed_y_std,event_id_speed_z_max,event_id_speed_z_mean,event_id_speed_z_min,event_id_speed_z_std,event_id_speed_xy_max,event_id_speed_xy_mean,event_id_speed_xy_min,event_id_speed_xy_std,event_id_speed_yz_max,event_id_speed_yz_mean,event_id_speed_yz_min,event_id_speed_yz_std,event_id_speed_zx_max,event_id_speed_zx_mean,event_id_speed_zx_min,event_id_speed_zx_std,event_id_px_max,event_id_px_mean,event_id_px_min,event_id_px_std,event_id_py_max,event_id_py_mean,event_id_py_min,event_id_py_std,event_id_pz_max,event_id_pz_mean,event_id_pz_min,event_id_pz_std,event_id_angle_xy_max,event_id_angle_xy_mean,event_id_angle_xy_min,event_id_angle_xy_std,event_id_angle_xz_max,event_id_angle_xz_mean,event_id_angle_xz_min,event_id_angle_xz_std,event_id_angle_yx_max,event_id_angle_yx_mean,event_id_angle_yx_min,event_id_angle_yx_std,event_id_angle_yz_max,event_id_angle_yz_mean,event_id_angle_yz_min,event_id_angle_yz_std,event_id_angle_zy_max,event_id_angle_zy_mean,event_id_angle_zy_min,event_id_angle_zy_std,event_id_angle_zx_max,event_id_angle_zx_mean,event_id_angle_zx_min,event_id_angle_zx_std,event_id_time_dis_max,event_id_time_dis_mean,event_id_time_dis_min,event_id_time_dis_std,event_id_time_x_max,event_id_time_x_mean,event_id_time_x_min,event_id_time_x_std,event_i

In [8]:
gc.collect()

20

In [9]:
# 用 np.ndarry 来做处理，使用 dataframe 会相当慢

jet_data = np.zeros(shape=(num_total_event, col_shape), dtype=np.float16)
len_commons = len(common_cols)
len_jets = len(jet_cols)

for row in tqdm(df.values):
    # 取出该行 jet 的 event_id
    event_id = row[-1]
    # n: 当前 event_id 已加入的 jet 个数
    # i: 当前 event_id 的序号
    n, i = di[event_id]
    # 如果当前 event_id 的 jet 个数少于 4 则加入进来
    if n < n_jets:
        # 首先是 common cols, 只在第一次时添加
        if n == 0:
            jet_data[i, 0:len_commons+len_jets] = row[0:len_commons+len_jets]
        else:
            jet_data[i, len_commons+n*len_jets:len_commons+(n+1)*len_jets] = row[len_commons:len_commons+len_jets]
        # particle 个数加 1
        di[event_id][0] += 1

100%|██████████| 1134555/1134555 [00:52<00:00, 21561.45it/s]


In [10]:
del df
gc.collect()

20

In [11]:
# 合并完的 jet 数据

df_jet = pd.DataFrame(jet_data)

In [12]:
df_jet.columns = common_cols + [f'j{i}_{j}' for i in range(n_jets) for j in jet_cols]
df_jet['event_id'] = event

In [13]:
df_jet.head()

,event_id_count,event_id_number_particles_max,event_id_number_particles_mean,event_id_number_particles_min,event_id_number_particles_std,event_id_mass_max,event_id_mass_mean,event_id_mass_min,event_id_mass_std,event_id_energy_max,event_id_energy_mean,event_id_energy_min,event_id_energy_std,event_id_mass_x_max,event_id_mass_x_mean,event_id_mass_x_min,event_id_mass_x_std,event_id_mass_y_max,event_id_mass_y_mean,event_id_mass_y_min,event_id_mass_y_std,event_id_mass_z_max,event_id_mass_z_mean,event_id_mass_z_min,event_id_mass_z_std,event_id_mass_xy_max,event_id_mass_xy_mean,event_id_mass_xy_min,event_id_mass_xy_std,event_id_mass_yz_max,event_id_mass_yz_mean,event_id_mass_yz_min,event_id_mass_yz_std,event_id_mass_zx_max,event_id_mass_zx_mean,event_id_mass_zx_min,event_id_mass_zx_std,event_id_energy_x_max,event_id_energy_x_mean,event_id_energy_x_min,event_id_energy_x_std,event_id_energy_y_max,event_id_energy_y_mean,event_id_energy_y_min,event_id_energy_y_std,event_id_energy_z_max,event_id_energy_z_mean,event_id_energy_z_min,event_id_energy_z_std,event_id_energy_xy_max,event_id_energy_xy_mean,event_id_energy_xy_min,event_id_energy_xy_std,event_id_energy_yz_max,event_id_energy_yz_mean,event_id_energy_yz_min,event_id_energy_yz_std,event_id_energy_zx_max,event_id_energy_zx_mean,event_id_energy_zx_min,event_id_energy_zx_std,event_id_particles_mass_max,event_id_particles_mass_mean,event_id_particles_mass_min,event_id_particles_mass_std,event_id_particles_energy_max,event_id_particles_energy_mean,event_id_particles_energy_min,event_id_particles_energy_std,event_id_distance_max,event_id_distance_mean,event_id_distance_min,event_id_distance_std,event_id_xy_dis_max,event_id_xy_dis_mean,event_id_xy_dis_min,event_id_xy_dis_std,event_id_yz_dis_max,event_id_yz_dis_mean,event_id_yz_dis_min,event_id_yz_dis_std,event_id_zx_dis_max,event_id_zx_dis_mean,event_id_zx_dis_min,event_id_zx_dis_std,event_id_x_div_dist_max,event_id_x_div_dist_mean,event_id_x_div_dist_min,event_id_x_div_dist_std,event_id_y_div_dist_max,event_id_y_div_dist_mean,event_id_y_div_dist_min,event_id_y_div_dist_std,event_id_z_div_dist_max,event_id_z_div_dist_mean,event_id_z_div_dist_min,event_id_z_div_dist_std,event_id_xy_div_dist_max,event_id_xy_div_dist_mean,event_id_xy_div_dist_min,event_id_xy_div_dist_std,event_id_yz_div_dist_max,event_id_yz_div_dist_mean,event_id_yz_div_dist_min,event_id_yz_div_dist_std,event_id_zx_div_dist_max,event_id_zx_div_dist_mean,event_id_zx_div_dist_min,event_id_zx_div_dist_std,event_id_speed_max,event_id_speed_mean,event_id_speed_min,event_id_speed_std,event_id_speed_x_max,event_id_speed_x_mean,event_id_speed_x_min,event_id_speed_x_std,event_id_speed_y_max,event_id_speed_y_mean,event_id_speed_y_min,event_id_speed_y_std,event_id_speed_z_max,event_id_speed_z_mean,event_id_speed_z_min,event_id_speed_z_std,event_id_speed_xy_max,event_id_speed_xy_mean,event_id_speed_xy_min,event_id_speed_xy_std,event_id_speed_yz_max,event_id_speed_yz_mean,event_id_speed_yz_min,event_id_speed_yz_std,event_id_speed_zx_max,event_id_speed_zx_mean,event_id_speed_zx_min,event_id_speed_zx_std,event_id_px_max,event_id_px_mean,event_id_px_min,event_id_px_std,event_id_py_max,event_id_py_mean,event_id_py_min,event_id_py_std,event_id_pz_max,event_id_pz_mean,event_id_pz_min,event_id_pz_std,event_id_angle_xy_max,event_id_angle_xy_mean,event_id_angle_xy_min,event_id_angle_xy_std,event_id_angle_xz_max,event_id_angle_xz_mean,event_id_angle_xz_min,event_id_angle_xz_std,event_id_angle_yx_max,event_id_angle_yx_mean,event_id_angle_yx_min,event_id_angle_yx_std,event_id_angle_yz_max,event_id_angle_yz_mean,event_id_angle_yz_min,event_id_angle_yz_std,event_id_angle_zy_max,event_id_angle_zy_mean,event_id_angle_zy_min,event_id_angle_zy_std,event_id_angle_zx_max,event_id_angle_zx_mean,event_id_angle_zx_min,event_id_angle_zx_std,event_id_time_dis_max,event_id_time_dis_mean,event_id_time_dis_min,event_id_time_dis_std,event_id_time_x_max,event_id_time_x_mean,event_id_time_x_min,event_id_time_x_std,event_i

In [14]:
del jet_data

gc.collect()

20

In [15]:
# 读入 event 数据

train_event = pd.read_csv('jet_complex_data/complex_train_R04_event.csv')

In [16]:
train_event.shape, df_jet.shape

((371377, 2), (371377, 1857))

In [17]:
df = pd.merge(df_jet, train_event, on=['event_id'])

In [18]:
gc.collect()

80

In [19]:
df['label'] = df['j0_label']
df = df.drop([f'j{i}_label' for i in range(n_jets)], axis=1)
df['label'] = df['label'].astype('int')

In [20]:
cols = [col for col in df.columns.tolist() if col not in ['number_of_jet_in_this_event', 'event_id', 'label']]
df = df.reindex(columns=['number_of_jet_in_this_event']+cols+['event_id', 'label'])
df.head()

,number_of_jet_in_this_event,event_id_count,event_id_number_particles_max,event_id_number_particles_mean,event_id_number_particles_min,event_id_number_particles_std,event_id_mass_max,event_id_mass_mean,event_id_mass_min,event_id_mass_std,event_id_energy_max,event_id_energy_mean,event_id_energy_min,event_id_energy_std,event_id_mass_x_max,event_id_mass_x_mean,event_id_mass_x_min,event_id_mass_x_std,event_id_mass_y_max,event_id_mass_y_mean,event_id_mass_y_min,event_id_mass_y_std,event_id_mass_z_max,event_id_mass_z_mean,event_id_mass_z_min,event_id_mass_z_std,event_id_mass_xy_max,event_id_mass_xy_mean,event_id_mass_xy_min,event_id_mass_xy_std,event_id_mass_yz_max,event_id_mass_yz_mean,event_id_mass_yz_min,event_id_mass_yz_std,event_id_mass_zx_max,event_id_mass_zx_mean,event_id_mass_zx_min,event_id_mass_zx_std,event_id_energy_x_max,event_id_energy_x_mean,event_id_energy_x_min,event_id_energy_x_std,event_id_energy_y_max,event_id_energy_y_mean,event_id_energy_y_min,event_id_energy_y_std,event_id_energy_z_max,event_id_energy_z_mean,event_id_energy_z_min,event_id_energy_z_std,event_id_energy_xy_max,event_id_energy_xy_mean,event_id_energy_xy_min,event_id_energy_xy_std,event_id_energy_yz_max,event_id_energy_yz_mean,event_id_energy_yz_min,event_id_energy_yz_std,event_id_energy_zx_max,event_id_energy_zx_mean,event_id_energy_zx_min,event_id_energy_zx_std,event_id_particles_mass_max,event_id_particles_mass_mean,event_id_particles_mass_min,event_id_particles_mass_std,event_id_particles_energy_max,event_id_particles_energy_mean,event_id_particles_energy_min,event_id_particles_energy_std,event_id_distance_max,event_id_distance_mean,event_id_distance_min,event_id_distance_std,event_id_xy_dis_max,event_id_xy_dis_mean,event_id_xy_dis_min,event_id_xy_dis_std,event_id_yz_dis_max,event_id_yz_dis_mean,event_id_yz_dis_min,event_id_yz_dis_std,event_id_zx_dis_max,event_id_zx_dis_mean,event_id_zx_dis_min,event_id_zx_dis_std,event_id_x_div_dist_max,event_id_x_div_dist_mean,event_id_x_div_dist_min,event_id_x_div_dist_std,event_id_y_div_dist_max,event_id_y_div_dist_mean,event_id_y_div_dist_min,event_id_y_div_dist_std,event_id_z_div_dist_max,event_id_z_div_dist_mean,event_id_z_div_dist_min,event_id_z_div_dist_std,event_id_xy_div_dist_max,event_id_xy_div_dist_mean,event_id_xy_div_dist_min,event_id_xy_div_dist_std,event_id_yz_div_dist_max,event_id_yz_div_dist_mean,event_id_yz_div_dist_min,event_id_yz_div_dist_std,event_id_zx_div_dist_max,event_id_zx_div_dist_mean,event_id_zx_div_dist_min,event_id_zx_div_dist_std,event_id_speed_max,event_id_speed_mean,event_id_speed_min,event_id_speed_std,event_id_speed_x_max,event_id_speed_x_mean,event_id_speed_x_min,event_id_speed_x_std,event_id_speed_y_max,event_id_speed_y_mean,event_id_speed_y_min,event_id_speed_y_std,event_id_speed_z_max,event_id_speed_z_mean,event_id_speed_z_min,event_id_speed_z_std,event_id_speed_xy_max,event_id_speed_xy_mean,event_id_speed_xy_min,event_id_speed_xy_std,event_id_speed_yz_max,event_id_speed_yz_mean,event_id_speed_yz_min,event_id_speed_yz_std,event_id_speed_zx_max,event_id_speed_zx_mean,event_id_speed_zx_min,event_id_speed_zx_std,event_id_px_max,event_id_px_mean,event_id_px_min,event_id_px_std,event_id_py_max,event_id_py_mean,event_id_py_min,event_id_py_std,event_id_pz_max,event_id_pz_mean,event_id_pz_min,event_id_pz_std,event_id_angle_xy_max,event_id_angle_xy_mean,event_id_angle_xy_min,event_id_angle_xy_std,event_id_angle_xz_max,event_id_angle_xz_mean,event_id_angle_xz_min,event_id_angle_xz_std,event_id_angle_yx_max,event_id_angle_yx_mean,event_id_angle_yx_min,event_id_angle_yx_std,event_id_angle_yz_max,event_id_angle_yz_mean,event_id_angle_yz_min,event_id_angle_yz_std,event_id_angle_zy_max,event_id_angle_zy_mean,event_id_angle_zy_min,event_id_angle_zy_std,event_id_angle_zx_max,event_id_angle_zx_mean,event_id_angle_zx_min,event_id_angle_zx_std,event_id_time_dis_max,event_id_time_dis_mean,event_id_time_dis_min,event_id_time_dis_std,event_id_time_x_max,event_id_time_x_mean,event_id_time_x_min

In [21]:
import re

c_cols = ['number_of_jet_in_this_event'] + [col for col in df.columns.tolist() if col.startswith('event_id_')]
p_cols = [col for col in df.columns.tolist() if re.match(r'j\d+_p\d+', col)]
j_cols = [col for col in df.columns.tolist() if col not in p_cols+c_cols+['event_id', 'label']]
df = df.reindex(columns=c_cols+j_cols+p_cols+['event_id', 'label'])
df.head()

,number_of_jet_in_this_event,event_id_count,event_id_number_particles_max,event_id_number_particles_mean,event_id_number_particles_min,event_id_number_particles_std,event_id_mass_max,event_id_mass_mean,event_id_mass_min,event_id_mass_std,event_id_energy_max,event_id_energy_mean,event_id_energy_min,event_id_energy_std,event_id_mass_x_max,event_id_mass_x_mean,event_id_mass_x_min,event_id_mass_x_std,event_id_mass_y_max,event_id_mass_y_mean,event_id_mass_y_min,event_id_mass_y_std,event_id_mass_z_max,event_id_mass_z_mean,event_id_mass_z_min,event_id_mass_z_std,event_id_mass_xy_max,event_id_mass_xy_mean,event_id_mass_xy_min,event_id_mass_xy_std,event_id_mass_yz_max,event_id_mass_yz_mean,event_id_mass_yz_min,event_id_mass_yz_std,event_id_mass_zx_max,event_id_mass_zx_mean,event_id_mass_zx_min,event_id_mass_zx_std,event_id_energy_x_max,event_id_energy_x_mean,event_id_energy_x_min,event_id_energy_x_std,event_id_energy_y_max,event_id_energy_y_mean,event_id_energy_y_min,event_id_energy_y_std,event_id_energy_z_max,event_id_energy_z_mean,event_id_energy_z_min,event_id_energy_z_std,event_id_energy_xy_max,event_id_energy_xy_mean,event_id_energy_xy_min,event_id_energy_xy_std,event_id_energy_yz_max,event_id_energy_yz_mean,event_id_energy_yz_min,event_id_energy_yz_std,event_id_energy_zx_max,event_id_energy_zx_mean,event_id_energy_zx_min,event_id_energy_zx_std,event_id_particles_mass_max,event_id_particles_mass_mean,event_id_particles_mass_min,event_id_particles_mass_std,event_id_particles_energy_max,event_id_particles_energy_mean,event_id_particles_energy_min,event_id_particles_energy_std,event_id_distance_max,event_id_distance_mean,event_id_distance_min,event_id_distance_std,event_id_xy_dis_max,event_id_xy_dis_mean,event_id_xy_dis_min,event_id_xy_dis_std,event_id_yz_dis_max,event_id_yz_dis_mean,event_id_yz_dis_min,event_id_yz_dis_std,event_id_zx_dis_max,event_id_zx_dis_mean,event_id_zx_dis_min,event_id_zx_dis_std,event_id_x_div_dist_max,event_id_x_div_dist_mean,event_id_x_div_dist_min,event_id_x_div_dist_std,event_id_y_div_dist_max,event_id_y_div_dist_mean,event_id_y_div_dist_min,event_id_y_div_dist_std,event_id_z_div_dist_max,event_id_z_div_dist_mean,event_id_z_div_dist_min,event_id_z_div_dist_std,event_id_xy_div_dist_max,event_id_xy_div_dist_mean,event_id_xy_div_dist_min,event_id_xy_div_dist_std,event_id_yz_div_dist_max,event_id_yz_div_dist_mean,event_id_yz_div_dist_min,event_id_yz_div_dist_std,event_id_zx_div_dist_max,event_id_zx_div_dist_mean,event_id_zx_div_dist_min,event_id_zx_div_dist_std,event_id_speed_max,event_id_speed_mean,event_id_speed_min,event_id_speed_std,event_id_speed_x_max,event_id_speed_x_mean,event_id_speed_x_min,event_id_speed_x_std,event_id_speed_y_max,event_id_speed_y_mean,event_id_speed_y_min,event_id_speed_y_std,event_id_speed_z_max,event_id_speed_z_mean,event_id_speed_z_min,event_id_speed_z_std,event_id_speed_xy_max,event_id_speed_xy_mean,event_id_speed_xy_min,event_id_speed_xy_std,event_id_speed_yz_max,event_id_speed_yz_mean,event_id_speed_yz_min,event_id_speed_yz_std,event_id_speed_zx_max,event_id_speed_zx_mean,event_id_speed_zx_min,event_id_speed_zx_std,event_id_px_max,event_id_px_mean,event_id_px_min,event_id_px_std,event_id_py_max,event_id_py_mean,event_id_py_min,event_id_py_std,event_id_pz_max,event_id_pz_mean,event_id_pz_min,event_id_pz_std,event_id_angle_xy_max,event_id_angle_xy_mean,event_id_angle_xy_min,event_id_angle_xy_std,event_id_angle_xz_max,event_id_angle_xz_mean,event_id_angle_xz_min,event_id_angle_xz_std,event_id_angle_yx_max,event_id_angle_yx_mean,event_id_angle_yx_min,event_id_angle_yx_std,event_id_angle_yz_max,event_id_angle_yz_mean,event_id_angle_yz_min,event_id_angle_yz_std,event_id_angle_zy_max,event_id_angle_zy_mean,event_id_angle_zy_min,event_id_angle_zy_std,event_id_angle_zx_max,event_id_angle_zx_mean,event_id_angle_zx_min,event_id_angle_zx_std,event_id_time_dis_max,event_id_time_dis_mean,event_id_time_dis_min,event_id_time_dis_std,event_id_time_x_max,event_id_time_x_mean,event_id_time_x_min

In [22]:
del df_jet, train_event

gc.collect()

20

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 371377 entries, 0 to 371376
Columns: 1855 entries, number_of_jet_in_this_event to label
dtypes: float16(1852), int64(2), object(1)
memory usage: 1.3+ GB


In [24]:
df.to_pickle('train_04j20p_fe_v3.pickle')

In [25]:
for i, col in enumerate(df.columns.tolist()):
    print(i, col)

0 number_of_jet_in_this_event
1 event_id_count
2 event_id_number_particles_max
3 event_id_number_particles_mean
4 event_id_number_particles_min
5 event_id_number_particles_std
6 event_id_mass_max
7 event_id_mass_mean
8 event_id_mass_min
9 event_id_mass_std
10 event_id_energy_max
11 event_id_energy_mean
12 event_id_energy_min
13 event_id_energy_std
14 event_id_mass_x_max
15 event_id_mass_x_mean
16 event_id_mass_x_min
17 event_id_mass_x_std
18 event_id_mass_y_max
19 event_id_mass_y_mean
20 event_id_mass_y_min
21 event_id_mass_y_std
22 event_id_mass_z_max
23 event_id_mass_z_mean
24 event_id_mass_z_min
25 event_id_mass_z_std
26 event_id_mass_xy_max
27 event_id_mass_xy_mean
28 event_id_mass_xy_min
29 event_id_mass_xy_std
30 event_id_mass_yz_max
31 event_id_mass_yz_mean
32 event_id_mass_yz_min
33 event_id_mass_yz_std
34 event_id_mass_zx_max
35 event_id_mass_zx_mean
36 event_id_mass_zx_min
37 event_id_mass_zx_std
38 event_id_energy_x_max
39 event_id_energy_x_mean
40 event_id_energy_x_min
41 e

1118 j1_p14_particle_category_abs
1119 j1_p14_particle_px
1120 j1_p14_particle_py
1121 j1_p14_particle_pz
1122 j1_p14_particle_energy
1123 j1_p14_particle_mass
1124 j1_p14_x_div_dist
1125 j1_p14_particle_distance
1126 j1_p14_particle_speed
1127 j1_p14_speed_x
1128 j1_p14_energy_x
1129 j1_p14_mass_x
1130 j1_p14_time_x
1131 j1_p14_angle_xy
1132 j1_p14_angle_xz
1133 j1_p15_particle_category
1134 j1_p15_particle_category_abs
1135 j1_p15_particle_px
1136 j1_p15_particle_py
1137 j1_p15_particle_pz
1138 j1_p15_particle_energy
1139 j1_p15_particle_mass
1140 j1_p15_x_div_dist
1141 j1_p15_particle_distance
1142 j1_p15_particle_speed
1143 j1_p15_speed_x
1144 j1_p15_energy_x
1145 j1_p15_mass_x
1146 j1_p15_time_x
1147 j1_p15_angle_xy
1148 j1_p15_angle_xz
1149 j1_p16_particle_category
1150 j1_p16_particle_category_abs
1151 j1_p16_particle_px
1152 j1_p16_particle_py
1153 j1_p16_particle_pz
1154 j1_p16_particle_energy
1155 j1_p16_particle_mass
1156 j1_p16_x_div_dist
1157 j1_p16_particle_distance
1158 